In [3]:
import numpy as np
import pandas as pd
import os
from scipy.integrate import quad_vec
CurrentPath = os.getcwd()
sndata = pd.read_csv(CurrentPath + '/2025/parsonage.txt', sep = ' ', engine='python')


1. Supernovae data:

$$M_B ≈ M_V ≈ −19.30 ± 0.03 + 5 \log(H_0/60) \qquad \text{Hillebrandt, W.; Niemeyer, J. C. (2000)}$$

$$\text{we will calculate} \ \log(d_L) \text{ with distance modulus } m - M = 5\log (d_L/10)$$
-> fit the log(d_L) with cosmological parameters


In [2]:

# observed data
mb = sndata['mb'].values
dmb = sndata['dmb'].values
z = sndata['zcmb'].values
H0 = 67.7 # km/s/Mpc Planck 2018
# distance modulus
absmb = -19.3 + 5*np.log10(H0/60) # H0 = 67.7 km/s/Mpc
dabsmb = 0.03

logdl_obs = (mb - absmb + 5)/5 
dlogdl_obs = (dmb + dabsmb)/5

In [ ]:
# calculated data via the model

# model parameters
# 1. Flat Lambda CDM
def integrand_flat_LCDM(z, Omega_m):
    Omega_L = 1 - Omega_m
    H0 = 67.7 # km/s/Mpc
    E = (1+z)**2*np.sqrt(Omega_m*(1+z) + Omega_L/(1+z)**2)
    return 1/E

def Flat_LCDM(z, Omega_m):
    Omega_L = 1 - Omega_m
    H0 = 67.7 # km/s/Mpc
    dp = integrand_flat_LCDM(z, Omega_m)
    return dl

# 2. Curvature Lambda CDM


# 3. 